In [ ]:
# import packages
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
import seaborn as sns


In [ ]:
#read data
train_features_data = pd.read_csv('../input/hr-dataset/train_LZdllcl.csv')
test_features_data = pd.read_csv('../input/hr-dataset/test_2umaH9m.csv')

In [ ]:
train_features_data.head()

In [ ]:
test_features_data.head()

In [ ]:
train_features_data.info()

In [ ]:
train_features_data.describe()

In [ ]:
train_features_data.drop(['employee_id'], axis="columns", inplace=True)
train_features_data.head()

In [ ]:
train_features_data.shape

In [ ]:
test_features_data.shape

**Categorical Features**

In [ ]:
cat_fetaures_col = []
for column in train_features_data.columns:
    if train_features_data[column].dtype == object:
        cat_fetaures_col.append(column)
        print(f"{column} : {train_features_data[column].unique()}")
        print(train_features_data[column].value_counts())
        print("-------------------------------------------")


**Numerical Features**

In [ ]:
#numeric-cat ==> discrete
disc_feature_col = []
for column in train_features_data.columns:
    if train_features_data[column].dtypes != object and train_features_data[column].nunique() <= 30:
        print(f"{column} : {train_features_data[column].unique()}")
        print(train_features_data[column].value_counts())
        disc_feature_col.append(column)
        print("-------------------------------------------")
        
disc_feature_col.remove('is_promoted')

In [ ]:
cont_feature_col=[]
for column in train_features_data.columns:
    if train_features_data[column].dtypes != object and train_features_data[column].nunique() > 30:
        print(f"{column} : Minimum: {train_features_data[column].min()}, Maximum: {train_features_data[column].max()}")
        cont_feature_col.append(column)
        print("-------------------------------------------")

In [ ]:
#there are missing values for "education" and "previous_year_rating" cols.
train_features_data.isnull().sum()

In [ ]:
#eliminate null values(fill with mode of that column)

for column in train_features_data.columns:
    train_features_data[column].fillna(train_features_data[column].mode()[0], inplace=True)


In [ ]:
print(train_features_data['education'].mode()[0])
print(train_features_data['previous_year_rating'].mode()[0])

In [ ]:
#there are no missing values in our dataset anymore!!!
train_features_data.isnull().sum()

In [ ]:
#there is no "NaN" values anymore
print(train_features_data['previous_year_rating'].unique())
print(train_features_data['education'].unique())

In [ ]:
train_features_data.info()

In [ ]:
#outlier analysis using box-plot(continuos data can have outliers(aykırı değerler))

sns.set(style="whitegrid",font_scale=1)
plt.figure(figsize=(7,7))
sns.boxplot(data=train_features_data[cont_feature_col])
plt.xticks(rotation=30)
plt.title("Box plot of continuos features")
plt.show()

In [ ]:
# find the IQR

q1 = train_features_data[cont_feature_col].quantile(.25)
q3 = train_features_data[cont_feature_col].quantile(.75)
IQR = q3-q1

print("         IQR")
print("------------------------------\n")
print(IQR)
print("         q1")
print("------------------------------\n")
print(q1)
print("         q3")
print("------------------------------\n")
print(q3)


lower_bound = q1 - 1.5*IQR
upper_bound = q3 + 1.5*IQR
print("\n--------lower bounds--------")
print(lower_bound)
print("\n--------upper bound---------")
print(upper_bound)


In [ ]:
print(lower_bound)
print("-------------------")
print(lower_bound[0])
print(lower_bound[1])
print(lower_bound[2])

In [ ]:
outliers_df = np.logical_or((train_features_data[cont_feature_col] < lower_bound), (train_features_data[cont_feature_col] > upper_bound)) 
outliers_df

In [ ]:
outlier_total=[]
outlier_percentage=[]

for col in list(train_features_data[cont_feature_col].columns):
    try:
        outlier_total.append(outliers_df[col].value_counts()[True])
        outlier_percentage.append((outliers_df[col].value_counts()[True] / outliers_df[col].value_counts().sum())*100)
    except:
        outlier_total.append(0)
        outlier_percentage.append(0)
        
print(outlier_total)
print(outlier_percentage)

outlier_number_df=pd.DataFrame(zip(list(outliers_df.columns), outlier_total,outlier_percentage), columns=['name', 'total', 'outlier(%)'])
#outlier_df.set_index('name', inplace=True)
outlier_number_df

In [ ]:
outlier_det_age_df=train_features_data[cont_feature_col]['age']
print(type(outlier_det_age_df))

outlier_det_los_df=train_features_data[cont_feature_col]['length_of_service']


In [ ]:
outlier_det_age_df

In [ ]:
print(outlier_det_age_df.head())
print(outlier_det_los_df.head())

In [ ]:
outlier_age=train_features_data[cont_feature_col]['age'] > upper_bound[0]
print(outlier_age.head())

outlier_los=train_features_data[cont_feature_col]['length_of_service'] > upper_bound[1]
print(outlier_los.head())



In [ ]:
print(outlier_age.nunique())
print(outlier_age.value_counts())

print(outlier_los.value_counts())


In [ ]:
#fill outliers upper bound

outlier_det_age_df[outlier_age]=upper_bound[0]
print(outlier_det_age_df[outlier_age])

print("=======================")
outlier_det_los_df[outlier_los]=upper_bound[1]
print(outlier_det_los_df[outlier_los])


In [ ]:
#update original train set with fixed outlier values

train_features_data['age']=outlier_det_age_df
train_features_data['length_of_service']=outlier_det_los_df

In [ ]:
#there is NO outlier anymore!!!(see the boxplot)

sns.set(style="whitegrid",font_scale=1)
plt.figure(figsize=(7,7))
sns.boxplot(data=train_features_data[cont_feature_col])
plt.xticks(rotation=30)
plt.title("Box plot of continuos features")
plt.show()

In [ ]:
"""
#baskilama(if value>upper --> make value=upper OR value<lower --> make value=lower)

outlier_det_age_df[outlier_age]=lower_bound[0]
outlier_det_age_df[outlier_age]

"""


In [ ]:
#encode ediyoruzzz!!!

#encoding categorical features (str-->float)

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()

enc.fit(train_features_data)
train_features_data_arr=enc.transform(train_features_data)

col_names_list=train_features_data.columns
encoded_categorical_df=pd.DataFrame(train_features_data_arr, columns=col_names_list)



In [ ]:
#check types
encoded_categorical_df.info()

In [ ]:
# split df to X and Y
from sklearn.model_selection import train_test_split

y = encoded_categorical_df.loc[:, 'is_promoted'].values
X = encoded_categorical_df.drop('is_promoted', axis=1)

# split data into 80-20 for training set / test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state=100)


In [ ]:
binary_cols = [col for col in list(encoded_categorical_df.columns) if encoded_categorical_df[col].nunique() <= 2] 
binary_cols.remove('is_promoted')

non_binary_cols = [col for col in list(encoded_categorical_df.columns) if encoded_categorical_df[col].nunique() > 2]

In [ ]:
#normalization(make all values bet. 0-1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X_train[non_binary_cols])

X_train_normalized_arr=scaler.transform(X_train[non_binary_cols])
X_train_normalized_df=pd.DataFrame(X_train_normalized_arr, columns=non_binary_cols)

X_test_normalized_arr=scaler.transform(X_test[non_binary_cols])
X_test_normalized_df=pd.DataFrame(X_test_normalized_arr, columns=non_binary_cols)

In [ ]:
X_train_normalized_df.head()

In [ ]:
X_test_normalized_df.head()

In [ ]:
X_train_binary_cols_df = X_train[binary_cols]
X_train_binary_cols_df.reset_index(inplace=True, drop=True)

X_train_final_df = pd.concat([X_train_binary_cols_df,X_train_normalized_df], axis=1)

X_train_final_df.head()

In [ ]:
X_test_binary_cols_df = X_test[binary_cols]
X_test_binary_cols_df.reset_index(inplace=True, drop=True)

X_test_final_df = pd.concat([X_test_binary_cols_df,X_test_normalized_df], axis=1)

X_test_final_df.head()

In [ ]:
#here is size of our train and test datasets
print(len(X_train_final_df))
print(len(X_test_final_df))

In [ ]:
print(len(y_train))
print(len(y_test))

In [ ]:
#import necessary libraries

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, roc_curve, confusion_matrix, classification_report, roc_auc_score
from sklearn.naive_bayes import GaussianNB, MultinomialNB, ComplementNB, CategoricalNB
from sklearn.svm import LinearSVC,SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# cross-validation with 10 splits
cv = StratifiedShuffleSplit(n_splits=10, random_state = 42, test_size=0.2)

In [ ]:
# display test scores and return result string and indexes of false samples
def display_test_scores(test, pred):
    str_out = ""
    str_out += ("TEST SCORES\n")
    str_out += ("=====================\n")
    
    #print accuracy
    accuracy = accuracy_score(test, pred)
    str_out += ("ACCURACY: {:.4f}\n".format(accuracy))
    str_out += ("\n")
    str_out += ("--------------------\n")
    

    #print AUC score
    auc = roc_auc_score(test, pred)
    str_out += ("AUC: {:.4f}\n".format(auc))
    str_out += ("\n")
    str_out += ("--------------------\n")

    #print confusion matrix
    str_out += ("CONFUSION MATRIX:\n")
    conf_mat = confusion_matrix(test, pred)
    str_out += ("{}".format(conf_mat))
    str_out += ("\n")
    str_out += ("\n")
    str_out += ("--------------------\n")
    
    #print FP, FN
    str_out += ("FALSE POSITIVES:\n")
    fp = conf_mat[1][0]
    pos_labels = conf_mat[1][0]+conf_mat[1][1]
    str_out += ("{} out of {} positive labels ({:.4f}%)\n".format(fp, pos_labels,fp/pos_labels))
    str_out += ("\n")
    str_out += ("------------------------------------\n")

    str_out += ("FALSE NEGATIVES:\n")
    fn = conf_mat[0][1]
    neg_labels = conf_mat[0][1]+conf_mat[0][0]
    str_out += ("{} out of {} negative labels ({:.4f}%)\n".format(fn, neg_labels, fn/neg_labels))
    str_out += ("\n")
    str_out += ("------------------------------------\n")

    #print classification report
    str_out += ("PRECISION, RECALL, F1 scores:\n\n")
    str_out += ("{}".format(classification_report(test, pred)))
    
    false_indexes = np.where(test != pred)
    return str_out, false_indexes

# Classifier-1: Decision Tree CART

In [ ]:

# CART decision tree
cart = DecisionTreeClassifier(random_state = 0)

# parameters 
parameters = {
                "criterion": ["gini","entropy"],
                "splitter": ["best","random"],
                "class_weight": [None, "balanced"],
                }

# grid search for parameters
grid_1 = GridSearchCV(estimator=cart, param_grid=parameters, cv=cv, n_jobs=-1)
grid_1.fit(X_train_final_df, y_train)

# print best scores
print("The best parameters are %s with a score of %0.4f"
      % (grid_1.best_params_, grid_1.best_score_))

# prediction results
y_pred = grid_1.predict(X_test_final_df)

# print accuracy metrics
results, false = display_test_scores(y_test, y_pred)
print(results)


# Classifier-2: Naive-Bayes

In [ ]:

# Naive-Bayes with different approaches
nb_list = [ GaussianNB(), MultinomialNB(), ComplementNB()]

for nb in nb_list:
    print("-----------", str(nb), "--------------")
    
    # parameters 
    parameters = {}

    # grid search for parameters
    grid_2 = GridSearchCV(estimator=nb, param_grid=parameters, cv=cv, n_jobs=-1)
    grid_2.fit(X_train_final_df, y_train)

    # print best scores
    print("The best parameters are %s with a score of %0.4f\n"
          % (grid_2.best_params_, grid_2.best_score_))

    # prediction results
    y_pred = grid_2.predict(X_test_final_df)

    # print accuracy metrics
    results, false = display_test_scores(y_test, y_pred)
    print(results)


# Classifier-3: SVM

In [ ]:
# SVM classifier
svm = SVC(tol=1e-5, random_state=0)

# parameters 
parameters = {
                'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                'C': [0.01, 0.1, 1, 10, 100],
                'max_iter': [100, 1000, 5000]
            }

# grid search for parameters
grid_3 = GridSearchCV(estimator=svm, param_grid=parameters, cv=cv, n_jobs=-1)
grid_3.fit(X_train_final_df, y_train)

# print best scores
print("The best parameters are %s with a score of %0.4f\n"
      % (grid_3.best_params_, grid_3.best_score_))

# prediction results
y_pred = grid_3.predict(X_test_final_df)

# print accuracy metrics
results, false = display_test_scores(y_test, y_pred)
print(results)


# Classifier-4: kNN 

In [ ]:

knn = KNeighborsClassifier()
# parameters 
parameters = {
                "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
                "n_neighbors": [5,15,25]
    }

# grid search for parameters
grid_4 = GridSearchCV(estimator=knn, param_grid=parameters, cv=cv, n_jobs=-1)
grid_4.fit(X_train_final_df, y_train)

# print best scores
print("The best parameters are %s with a score of %0.4f\n"
      % (grid_4.best_params_, grid_4.best_score_))

# prediction results
y_pred = grid_4.predict(X_test_final_df)

# print accuracy metrics
results, false = display_test_scores(y_test, y_pred)
print(results)


# Classifier-5: Logistic Regression 

In [ ]:


logit = LogisticRegression(random_state=0)
# parameters 
parameters = {
                "penalty":['l1', 'l2'],
                "C": [0.01, 0.1, 1, 10, 100],
                "max_iter": [100,1000,5000],
             }

# grid search for parameters
grid_5 = GridSearchCV(estimator=logit, param_grid=parameters, cv=cv, n_jobs=-1)
grid_5.fit(X_train_final_df, y_train)

# print best scores
print("The best parameters are %s with a score of %0.4f\n"
      % (grid_5.best_params_, grid_5.best_score_))

# prediction results
y_pred = grid_5.predict(X_test_final_df)

# print accuracy metrics
results, false = display_test_scores(y_test, y_pred)
print(results)



# Classifier-6:RF

In [ ]:

rf = RandomForestClassifier(random_state=0)

# parameters 
parameters = {
                "bootstrap": ["True","False"],
                "max_features": [None, "sqrt", "log2"],
                "class_weight": [None, "balanced", "balanced_subsample"],
                "max_samples": [None, 0.3, 0.5, 0.7, 0.9],
                "n_estimators": [10, 100, 200]
                
}

# grid search for parameters
grid_6 = GridSearchCV(estimator=rf, param_grid=parameters, cv=cv, n_jobs=-1)
grid_6.fit(X_train_final_df, y_train)

# print best scores
print("The best parameters are %s with a score of %0.4f"
      % (grid_6.best_params_, grid_6.best_score_))

# prediction results
y_pred = grid_6.predict(X_test_final_df)

# print accuracy metrics
results, false = display_test_scores(y_test, y_pred)
print(results)



# Classifier-7: Bagging Meta Estimator

In [ ]:


from sklearn.ensemble import BaggingClassifier

bag = BaggingClassifier(random_state=0)

# parameters 
parameters = {
                "bootstrap": ["True","False"],
                "max_features": [0.3, 0.5, 0.7, 0.9, 1],
                "max_samples": [0.3, 0.5, 0.7, 0.9],
                "n_estimators": [10, 100, 200]
                
}

# grid search for parameters
grid_7 = GridSearchCV(estimator=bag, param_grid=parameters, cv=cv, n_jobs=-1)
grid_7.fit(X_train_final_df, y_train)

# print best scores
print("The best parameters are %s with a score of %0.4f"
      % (grid_7.best_params_, grid_7.best_score_))

# prediction results
y_pred = grid_7.predict(X_test_final_df)

# print accuracy metrics
results, false = display_test_scores(y_test, y_pred)
print(results)


# Classifier-8: ExtraTressClassifier 

In [ ]:

from sklearn.ensemble import ExtraTreesClassifier

extra = ExtraTreesClassifier(random_state=0)

# parameters 
parameters = {
                "bootstrap": ["True","False"],
                "max_features": [None, "sqrt", "log2"],
                "class_weight": [None, "balanced", "balanced_subsample"],
                "max_samples": [None, 0.3, 0.5, 0.7, 0.9],
                "n_estimators": [10, 100, 200]
                
}

# grid search for parameters
grid_8 = GridSearchCV(estimator=extra, param_grid=parameters, cv=cv, n_jobs=-1)
grid_8.fit(X_train_final_df, y_train)

# print best scores
print("The best parameters are %s with a score of %0.4f"
      % (grid_8.best_params_, grid_8.best_score_))

# prediction results
y_pred = grid_8.predict(X_test_final_df)

# print accuracy metrics
results, false = display_test_scores(y_test, y_pred)
print(results)
